In [15]:
import wandb
import json
import pandas as pd
import numpy as np

In [16]:
api = wandb.Api(timeout=120)

In [36]:
with open('data/best_parents.json') as f:
    files = json.load(f)

for env_id, alg_dict in files.items():
    print(f'Processing {env_id}')
    for alg, (_, runs) in alg_dict.items():
        print(f'Processing {alg}')
        for run_path in runs:
            run = api.run(run_path)
            seed = run.config['seed']
            if 'front' in run.summary:
                front = np.array(run.summary['front'])
            else:
                output = run.file('output.log')
                file = output.download(root=f'data/{env_id}/{alg}', exist_ok=True)
                path = file.name

                front = []
                # Open as a text file
                with open(path) as f:
                    # Read the file contents and generate a list of lines
                    lines = f.readlines()
                    for line in lines:
                        if 'Found' in line:
                            split = line.split('[')[2].split(']')[0].split(' ')
                            vec = []
                            for val in split:
                                if not val == '':
                                    vec.append(float(val))
                            front.append(vec)
                front = np.array(front)
                run.summary['front'] = front.tolist()
                run.update()

            num_objectives = front.shape[1]
            columns = [f'obj_{i}' for i in range(num_objectives)]
            df = pd.DataFrame(front, columns=columns)

            # Save as a CSV to my directory
            new_path = f'fronts/{env_id}/{alg}_{seed}.csv'
            df.to_csv(new_path, index=False)

Processing deep-sea-treasure-concave-v0
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
Processing minecart-v0
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
Processing mo-reacher-v4
Processing SN-MO-PPO
Processing SN-MO-DQN
Processing SN-MO-A2C
